In [1]:
#!pip install pyunpack wget patool plotly cufflinks gitpython


- AI Hub Notebook: [AI Hub - Interpretable Multi-Horizon Time Series Forecasting with TFT](https://aihub.cloud.google.com/p/products%2F9f39ad8d-ad81-4fd9-8238-5186d36db2ec)


In [ ]:
%tensorflow_version 1.x


In [ ]:
import os
import pprint

In [ ]:
if(DEVICE == 'tpu'):  
    if 'COLAB_TPU_ADDR' not in os.environ:
        print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
    else:
        tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
        print ('TPU address is', tpu_address)

        with tf.Session(tpu_address) as session:
            devices = session.list_devices()

        print('TPU devices:')
        pprint.pprint(devices)

if(DEVICE == 'gpu'):
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        raise SystemError('GPU device not found')
    print('Found GPU at: {}'.format(device_name)) 

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
    print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
    print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
    print('re-execute this cell.')
else:
    print('You are using a high-RAM runtime!')

In [ ]:
# import IPython
# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

In [ ]:

from git import Repo

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')



In [ ]:
assets_dir = '/content/drive/My Drive/Colab Notebooks/od/assets'
repo_dir = assets_dir + '/repo'
print(repo_dir)
if not os.path.exists(repo_dir):
    os.makedirs(repo_dir)

In [ ]:
if not os.listdir(repo_dir):
    git_url = "https://github.com/google-research/google-research.git"
    Repo.clone_from(git_url, repo_dir)
    
# Sets current directory
tft_dir = os.path.join(repo_dir, 'tft')
os.chdir(tft_dir)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
from script_download_data import main as download_data

In [ ]:
# Download Parameters
expt_name = 'traffic'
output_folder = os.path.join(os.getcwd(), 'outputs')
force_download = False

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
csv_file = os.path.join(output_folder, 'data', 'traffic', 'hourly_data.csv')
if not os.path.exists(csv_file):
    download_data(expt_name, force_download=True, output_folder=output_folder)
df = pd.read_csv(csv_file, index_col=0)

In [ ]:
from data_formatters.base import GenericDataFormatter, DataTypes, InputTypes

print('Available data types:')
for option in DataTypes:
    print(option)
    
print()
print('Available input types')
for option in InputTypes:
    print(option)

In [ ]:
from libs import utils
import sklearn.preprocessing 

### TrafficFormatter
- This class defines and formats data for traffic dataset
- It performs z-score normalization across the entire dataset
- z-score normalization ensures reusable functions


In [ ]:
class TrafficFormatters(GenericDataFormatter):
    
    # This defines the types used by each column
    _column_definition = [
        ('id', DataTypes.REAL_VALUED, InputTypes.ID),
        ('hours_from_start', DataTypes.REAL_VALUED, InputTypes.TIME),
        ('values', DataTypes.REAL_VALUED, InputTypes.TARGET),
        ('time_on_day', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
        ('day_of_week', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
        ('hours_from_start', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
        ('categorical_id', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT),
    ]
    
    def split_data(self, df, valid_boundary=151, test_boundary=166):
        """
        Splits dataframe into train, validation and test frames.
        This also calibrates scaling object and transforms data for each split
        
        Args:
            df: Source data frame to split
            valid_boundary: Starting year for validation data
            test_boundary: Starting year for test data
        
        Returns:
            Tuple of transformed tvt data
        """
        print('Formatting tvt splits')
        
        index = df['sensor_day']
        train = df.loc[index < valid_boundary]
        valid = df.loc[(index >= valid_boundary - 7) & (index < test_boundary)]
        test = df.loc[index >= test_boundary - 7]
        
        self.set_scalers(train)
        
        return (self.transform_inputs(data) for data in [train, valid, test])
    
    def set_scalers(self, df):
        """
        Calibrates scalers using the data df supplied
        """
        print('Setting scalers with training data')
        
        column_definitions = self.get_column_definition()
        id_column = utils.get_single_col_by_input_type(InputTypes.ID, column_definition=column_definitions)
        target_column = utils.get_single_col_by_input_type(InputTypes.TARGET, column_definitions)
        
        # Extract identifiers in case required
        self.identifiers = list(df[id_column].unique())
        
        # Format real scalers
        real_inputs = utils.extract_cols_from_data_type(
            DataTypes.REAL_VALUED,
            column_definitions,
            {
                InputTypes.ID, InputTypes.TIME
            }
        )
        
        data = df[real_inputs].values
        self._real_scalers = sklearn.preprocessing.StandardScaler().fit(data)
        self._target_scaler = sklearn.preprocessing.StandardScaler().fit(df[[target_column]].values)
        
        # Format categorical scalers
        categorical_inputs = utils.extract_cols_from_data_type(
            DataTypes.CATEGORICAL, column_definitions,
            {
                InputTypes.ID, InputTypes.TIME
            }
        )
        
        categorical_scalers = {}
        num_classes = []
        
        for col in categorical_inputs:
            # Set all to str so that we dont have mixed integer/string columns
            srs = df[col].apply(str)
            categorical_scalers[col] = sklearn.preprocessing.LabelEncoder().fit(srs.values)
            num_classes.append(srs.nunique())
            
        # Set categorical scaler ouputs
        self._cat_scalers = categorical_scalers
        self._num_classes_per_cat_input = num_classes
        
    def transform_inputs(self, df):
        """
        Performs features transformations
        
        This includes both feature engineering, pre-processing and normalization
        """
        
        output = df.copy()
        
        if(self._real_scalers is None and self._cat_scalers is None):
            raise ValueError('Scalers have not been set!')
            
        column_definitions = self.get_column_definition()
        
        real_inputs = utils.extract_cols_from_data_type(
            DataTypes.REAL_VALUED, column_definitions, {
                InputTypes.ID,
                InputTypes.TIME
            }
        )
        
        
        output[real_inputs] = self._real_scalers.transform(
            df[real_inputs].values
        )
        
        categorical_inputs = utils.extract_cols_from_data_type(
            DataTypes.CATEGORICAL, column_definitions,
            {
                InputTypes.ID, InputTypes.TIME
            }
        )
        
        # Format Categorical Inputs
        for col in categorical_inputs:
            string_df = df[col].apply(str)
            output[col] = self._cat_scalers[col].transform(string_df)
            
        return output
    
    def format_predictions(self, predictions):
        """
        Reverts any normalization to give predictions in orignal scale
        
        Args:
            predictions: DF or model predictions
            
        Returns:
            DF of unnormalized predictions
        """
        
        output = predictions.copy()
        
        column_names = predictions.columns
        
        for col in column_names:
            if col not in {'forecast_time', 'identifier'}:
                output[col] = self._target_scaler.inverse_transform(predictions[col])
                
        return output
    
    def get_fixed_params(self):
        """
        Returns fixed model parameters for experiments
        """
        
        fixed_params = {
            'total_time_steps': 8*24, # Total width of the Temporal Fusion Decoder
            'num_encoder_steps': 7*24, # Length of LSTM decoder (ie # historical inputs)
            'num_epochs': 50, # 100,
            'early_stopping_patience': 5, # Early stopping threshold for # iterations with no loss improvement
            'multiprocessing_workers': 5 # Number of multi-processing workers
        }
        return fixed_params
    
    

In [ ]:
### Explore

df.head()

### Training and Evaluating the TFT
Using the data formatting in TrafficFormatter, we next walk through the procedure for training the TFT

First, we get all data-related parameters from the data formatter and define TFT model parameters

In [ ]:
# Create a data formatter
data_formatter = TrafficFormatters()

# Split Data
train, valid, test = data_formatter.split_data(df)

print(f'Train: {train.shape}, Test: {test.shape}, Val: {valid.shape}')

train.head()

In [ ]:
data_params = data_formatter.get_experiment_params()
data_params

In [ ]:
# Model parameters for calbration
model_params = {
    'dropout_rate': 0.3, # Dropout discard rate
    'hidden_layer_size': 320, # Internal state size of TFT
    'learning_rate': 0.001, # ADAM initial learning rate
    'minibatch_size': 128, # Minibatch size of training
    'max_gradient_norm': 100.0, # Max norm for gradient clipping
    'num_heads': 4, # Number of heads for multi-head attention
    'stack_size': 1 # Number of stacks (default 1 for interpretability)
}

# Folder to save network weights during training
model_folder = os.path.join(output_folder, 'saved_models', 'traffic', 'fixed')
model_params['model_folder'] = model_folder

model_params.update(data_params)

In [ ]:
#import tensorflow as tf
from libs.tft_model import TemporalFusionTransformer

In [ ]:
tf_config = utils.get_default_tensorflow_config(tf_device=DEVICE, gpu_id=0)

In [ ]:
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:
    tf.keras.backend.set_session(sess)
    model = TemporalFusionTransformer(model_params, use_cudnn=USE_CUDNN)
    
    if(not model.training_data_cached()):
        model.cache_batched_data(train, "train", num_samples=450000)
        model.cache_batched_data(valid, "valid", num_samples=50000)

    print("Data acquisition completed, training starts...")
    model.fit()
    model.save(model_folder)

In [ ]:
!ls /content/repo/tft/outputs/saved_models/
!zip -r /content/repo/tft/outputs/saved_models.zip /content/repo/tft/outputs/saved_models

In [ ]:
tf.reset_default_graph()
with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:
    tf.keras.backend.set_session(sess)
    
    # Create a new model and load weights
    model = TemporalFusionTransformer(model_params, use_cudnn=USE_CUDNN)
    model.load(model_folder)
    
    # Make forecasts
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map['targets'])
    
    targets = data_formatter.format_predictions(output_map['targets'])
    
    # Format predictions
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])
    
    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns"""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p50_loss = utils.numpy_normalised_quantile_loss(
        extract_numerical_data(targets), extract_numerical_data(p50_forecast),
        0.5
    )
    p90_loss = utils.numpy_normalised_quantile_loss(
        extract_numerical_data(targets), extract_numerical_data(p90_forecast),
        0.9
    )
    
    print("Normalised quantile losses: P50={}, P90={}".format(p50_loss.mean(), p90_loss.mean()))


### Interpretability Use Cases
The relationships learnt by TFT can also be studied using the trained model, throug
- Analyzing the variable selection weights to identify significant features for the prediction problem
- Visualizing distributions of self-attention weights to determine the presence of any persistent temporal relationships

In the remainder of this section, we demonstrate two interpretability use cases to showcase the above

### Generate Weights for Interpretability
First, we generate all necessary variable selection and attention weights required for analysis

In [ ]:
# Store outputs in maps 
counts = 0
interpretability_weights = {
    k: None for k in [
        'decoder_self_attn', 'static_flags', 'historical_flags', 'future_flags'
    ]
}

tf.reset_default_graph()
with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:
    tf.keras.backend.set_session(sess)
    
    # Create a new model and load weights
    model = TemporalFusionTransformer(model_params, use_cudnn=USE_CUDNN)
    for identifier, sliced in test.groupby('id'):
        print(f"Getting attention weights for {identifier}")
        weights = model.get_attention(sliced)
        
        for k in interpretability_weights:
            w = weights[k]
            
            # Average attention across heads if necessary
            if(k == 'decoder_self_attn'):
                w = w.mean(axis=0)
                
                # Store a single matrix for weights to reduce memory footprint
                batch_size, _, _ = w.shape
                counts += batch_size
                
            if(interpretability_weights[k] is None):
                interpretability_weights[k] = w.sum(axis=0)
            else:
                interpretability_weights[k] += w.sum(axis=0)
                
interpretability_weight = {
    k: interpretability_weights[k] / counts for k in interpretability_weights
}

print('Done.')

### Use Case 1: Analysing Variable Importance
- Analyze the distribution of variable selection weights on the input layer
- Use it to quantify the relative importance of a given feature for the prediction problem in general
- This is split into variable importance for static covariates, time-varying historical inputs and known future inputs as shown below

In [ ]:
import numpy as np
def get_range(static_gate, axis=None):
    """
    Return the mean, 10th, 50th and 90th percentile of variable importance weights
    """
    return {
        'Mean': static_gate.mean(axis=axis),
        '10%': np.quantile(static_gate, 0.1, axis=axis),
        '25%': np.quantile(static_gate, 0.25, axis=axis),
        '50%': np.quantile(static_gate, 0.5, axis=axis),
        '1SD, 67%': np.quantile(static_gate, 0.67, axis=axis),
        '90%': np.quantile(static_gate, 0.9, axis=axis),
        '2SD 99.7%': np.quantile(static_gate, 0.977, axis=axis),
    }

**Static Variable Importance**

In [ ]:
def flatten(x):
    static_attn = x
    static_attn = static_attn.reshape([-1, static_attn.shape[-1]])
    return static_attn

static_attn = flatten(interpretability_weights['static_flags'])
m = get_range(static_attn, axis=0)
pd.DataFrame({
    k: pd.Series(m[k], index=['ID']) for k in m
})

**Temporal Variable Importance - Past Inputs**

In [ ]:
x = flatten(interpretability_weights['historical_flags'])
m = get_range(x, axis=0)
pd.DataFrame({
    k: pd.Series(m[k], index=['Hour of Day', 'Day of Week', 'Time Index', 'Target']) for k in m
})

**Temporal Variable Importance -- Future Inputs**

In [ ]:
x = flatten(interpretability_weights['future_flags'])
m = get_range(x, axis=0)
pd.DataFrame({
     k: pd.Series(m[k], index=['Hour of Day', 'Day of Week', 'Time Index']) for k in m
})

### Use Case 2: Visualizing Persistent Temporal Patterns
- We analyze the distribution of self-attention weights across various horizons to see if any persistent temporal patterns exist within the dataset
- Through which, identify any seasonal patterns or lagged relationships in the dataset,
- Based on that past time steps are consistently important for predictions at a given horizon

We visualize this using the average attention pattern for various prediction horizons 

**Mean Attention Weights for Various Prediction Horizons**

In [ ]:
# Plotting libraries and functions
import plotly.offline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot
import plotly.graph_objs as go
import cufflinks as cf
from IPython.display import HTML

In [ ]:
# Loads plotly charts
def iplot(fig, s='plot.html'):
    filename = os.path.join(output_folder, s)
    plotly.offline.plot(fig, filename=filename, auto_open=False)
    return HTML(filename)

def plotly_chart(
    df, title=None, kind='scatter', 
    x_label=None, y_label=None, secondary_y=None, 
    fill=None, shape=None, subplots=False
):
    fig = df.iplot(
        asFigure=True, title=title, kind=kind,
        xTitle=x_label, yTitle=y_label, secondary_y=secondary_y,
        fill=fill, subplots=subplots, shape=shape
    )
    
    return iplot(fig)



In [ ]:
self_attn = interpretability_weights['decoder_self_attn']

means = pd.DataFrame({
    "horizon={}".format(k): self_attn[model.num_encoder_steps + k - 1, :] for k in [1, 5, 10, 15, 20]
})
means.index -= model.num_encoder_steps

plotly_chart(
    means,
    x_label="Position Index (n)",
    y_label="Mean Attention Weight",
    title="Average Attention Pattern at Various Prediction Horizons"
)